In [1]:
# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install("TargetDecoy")

# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")

# BiocManager::install("mzID")

In [2]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1001. InVitroWildfire EVProteomics/P1001.3. Analyses/P1001.3.4. Data Preprocessing/Input')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1001. InVitroWildfire EVProteomics/P1001.3. Analyses/P1001.3.4. Data Preprocessing/Output')
cur_date = "061122"

library(readxl)
library(tidyverse)
library(mzID)
library(TargetDecoy)
library(imputeLCMD)
library(stats)

#reading in files
identifiers_df = data.frame(read_excel("Identifiers_030422.xlsx"))
set1_proteome_df = data.frame(read_excel("PC927-Rager_LFQProteome_Results_RawValues_R ready.xlsx", sheet = 1))[,c(1,6,22:35)]
set2_proteome_df = data.frame(read_excel("PC927-Rager_LFQProteome_Results_RawValues_R ready.xlsx", sheet = 2))[,c(1,6,22:35)]

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘mzID’


The following object is masked from ‘package:dplyr’:

    id


The following object is masked from ‘package:purrr’:

    flatten


Warning message in fun(libname, pkgname):
“mzR has been built against a different Rcpp version (1.0.8)
than is installed on your system (1.0.8.3). This might lead to errors
when loading mzR. If you encounter such issues, please send a report,
including the output of sessionInfo() to the Bioc support forum at 
https://support.bioconductor.org/. For details see also
https://github.com/sneumann/mzR/wiki/mzR-Rcpp-compiler-linker-issue.”
Loading required 

In [3]:
head(identifiers_df)
head(set1_proteome_df)
head(set2_proteome_df)

,IDs_from_Core,Our_Analysis_ID,Set
,<chr>,<chr>,<chr>
1,Abundance..Pooled.1,CoCulture_PooledSample1,Set1
2,Abundance..Pooled.2,CoCulture_PooledSample2,Set1
3,Abundance..1..Vehicle,CoCulture_VehicleCT1,Set1
4,Abundance..3..Vehicle,CoCulture_VehicleCT2,Set1
5,Abundance..5..Vehicle,CoCulture_VehicleCT3,Set1
6,Abundance..7..Vehicle,CoCulture_VehicleCT4,Set1


,Accession,Number_of_UniquePeptides,Abundance..Pooled.1,Abundance..Pooled.2,Abundance..1..Vehicle,Abundance..3..Vehicle,Abundance..5..Vehicle,Abundance..7..Vehicle,Abundance..9..Vehicle,Abundance..11..Vehicle,Abundance..2..Wood.Smoke,Abundance..4..Wood.Smoke,Abundance..6..Wood.Smoke,Abundance..8..Wood.Smoke,Abundance..10..Wood.Smoke,Abundance..12..Wood.Smoke
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,P02769,99,4.89e+11,4.43e+11,6.26e+11,4.38e+11,4.74e+11,3.40e+11,4.94e+11,4.85e+11,4.54e+11,6.23e+11,7.37e+11,5.13e+11,3.96e+11,6.86e+11
2,P02751,201,2.73e+11,2.41e+11,2.24e+11,2.43e+11,3.24e+11,2.53e+11,2.45e+11,1.35e+11,1.68e+11,2.37e+11,2.98e+11,2.54e+11,1.97e+11,2.50e+11
3,P12763,34,2.63e+11,2.29e+11,3.10e+11,2.33e+11,2.70e+11,2.42e+11,2.71e+11,2.50e+11,2.67e+11,2.86e+11,3.27e+11,2.63e+11,2.52e+11,3.03e+11
4,Q2UVX4,162,5.65e+10,5.15e+10,6.76e+10,4.31e+10,4.09e+10,4.12e+10,4.92e+10,3.53e+10,4.76e+10,4.01e+10,3.68e+10,6.53e+10,4.45e+10,4.69e+10
5,P98160,210,2.27e+10,2.04e+10,3.01e+10,1.79e+10,1.40e+10,2.64e+10,2.81e+10,2.85e+10,1.97e+10,1.40e+10,1.30e+10,2.19e+10,2.46e+10,2.27e+10
6,P12111,201,2.15e+10,2.03e+10,1.55e+10,2.12e+10,2.29e+10,1.99e+10,2.00e+10,9.49e+09,1.67e+10,2.09e+10,2.67e+10,2.63e+10,2.06e+10,2.60e+10


,Accession,Number_of_UniquePeptides,Abundance..Pooled.1,Abundance..Pooled.2,Abundance..13..Vehicle,Abundance..15..Vehicle,Abundance..17..Vehicle,Abundance..19..Vehicle,Abundance..21..Vehicle,Abundance..23..Vehicle,Abundance..14..Wood.Smoke,Abundance..16..Wood.Smoke,Abundance..18..Wood.Smoke,Abundance..20..Wood.Smoke,Abundance..22..Wood.Smoke,Abundance..24..Wood.Smoke
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,P02751,193,197908273327,215449909749,290377149672,237808533258,300713328573,246820946145,304313034954,264966884310,243333335207,199860505414,279706283336,225170612912,229939459846,154501914723
2,P02769,97,342657042076,370488835120,308967859550,242917660277,278096520636,259674859546,291397652436,563821412595,281882492897,732144132018,248045691869,233383724786,349312465553,251094645011
3,P07996,34,64580519826,73778236233,181775619913,129485605438,169377550217,134685992120,114143664541,249366551496,132714564698,128084639698,112262919453,111419806864,73857936963,108923410421
4,Q2UVX4,160,41255020284,45525199756,55681501380,35618922477,44222632333,47245712210,44043324866,75943777589,53160804546,68298716794,48638631002,47654755526,35704911241,52145684636
5,P98160,207,16164372416,18035124503,20257643320,13577050138,17442737821,24582420164,20491703047,36081485773,20355674908,16277580495,15654918570,19579785521,16823558423,20030021253
6,P12763,32,180364995691,186934859645,159857253388,143580491033,158608208515,140452510019,145834623969,220324446614,137520441019,237042125375,156248949606,124984210103,171103871588,143324724008


In [4]:
# creating 1 proteome df
set1_proteome_df = set1_proteome_df %>%
    pivot_longer(cols = 3:16, names_to = "IDs_from_Core", values_to = "Value") %>%
    mutate(Set = c("Set1"))

set2_proteome_df = set2_proteome_df %>%
    pivot_longer(cols = 3:16, names_to = "IDs_from_Core", values_to = "Value") %>%
    mutate(Set = c("Set2"))

proteome_df = rbind(set1_proteome_df, set2_proteome_df) %>%
    rename(Protein = Accession)
head(proteome_df)

Protein,Number_of_UniquePeptides,IDs_from_Core,Value,Set
<chr>,<dbl>,<chr>,<dbl>,<chr>
P02769,99,Abundance..Pooled.1,4.89e+11,Set1
P02769,99,Abundance..Pooled.2,4.43e+11,Set1
P02769,99,Abundance..1..Vehicle,6.26e+11,Set1
P02769,99,Abundance..3..Vehicle,4.38e+11,Set1
P02769,99,Abundance..5..Vehicle,4.74e+11,Set1
P02769,99,Abundance..7..Vehicle,3.40e+11,Set1


In [5]:
#adding in our own ids
proteome_df = inner_join(identifiers_df, proteome_df) %>%
    select(-IDs_from_Core) %>%
    rename(ID = Our_Analysis_ID, Unique_Peptides = Number_of_UniquePeptides) %>%
    #creating a col that denotes the general treatment to remove proteins within those treatment groups
    mutate(Treatment = ifelse(grepl("CoCulture_PooledSample", ID), "CoCulture_PooledSample",
                             ifelse(grepl("CoCulture_Vehicle", ID), "CoCulture_Vehicle",
                                   ifelse(grepl("CoCulture_Exposed", ID), "CoCulture_Exposed",
                                         ifelse(grepl("TriCulture_PooledSample", ID), "TriCulture_PooledSample",
                                             ifelse(grepl("TriCulture_Vehicle", ID), "TriCulture_Vehicle",
                                                   ifelse(grepl("TriCulture_Exposed", ID), "TriCulture_Exposed",
                                         
                                         NA)))))))
head(proteome_df)

Joining, by = c("IDs_from_Core", "Set")


,ID,Set,Protein,Unique_Peptides,Value,Treatment
,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,CoCulture_PooledSample1,Set1,P02769,99,4.89e+11,CoCulture_PooledSample
2,CoCulture_PooledSample1,Set1,P02751,201,2.73e+11,CoCulture_PooledSample
3,CoCulture_PooledSample1,Set1,P12763,34,2.63e+11,CoCulture_PooledSample
4,CoCulture_PooledSample1,Set1,Q2UVX4,162,5.65e+10,CoCulture_PooledSample
5,CoCulture_PooledSample1,Set1,P98160,210,2.27e+10,CoCulture_PooledSample
6,CoCulture_PooledSample1,Set1,P12111,201,2.15e+10,CoCulture_PooledSample


# 1. Peptide Normalization

Normalize to total amount of peptide per sample: sum the abundance values for each sample, then calculate the median value across all of the summed abundance values, and calculate the ratio

In [6]:
normalized_df = proteome_df %>%
    group_by(Set, ID) %>%
    #removing missing data
    drop_na() %>%
    mutate(Summed_Value = sum(Value)) %>%
    #calculating median across all samples but still within each set
    ungroup() %>%
    group_by(Set) %>%
    mutate(Median_of_Sum = median(Summed_Value), Norm_Factor = Summed_Value/ Median_of_Sum, 
           Norm_Value = Value/Norm_Factor) %>%
    select(-c("Summed_Value", "Median_of_Sum", "Norm_Factor"))
head(normalized_df)

ID,Set,Protein,Unique_Peptides,Value,Treatment,Norm_Value
<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>
CoCulture_PooledSample1,Set1,P02769,99,4.89e+11,CoCulture_PooledSample,430002659858
CoCulture_PooledSample1,Set1,P02751,201,2.73e+11,CoCulture_PooledSample,240062834645
CoCulture_PooledSample1,Set1,P12763,34,2.63e+11,CoCulture_PooledSample,231269324218
CoCulture_PooledSample1,Set1,Q2UVX4,162,5.65e+10,CoCulture_PooledSample,49683333910
CoCulture_PooledSample1,Set1,P98160,210,2.27e+10,CoCulture_PooledSample,19961268668
CoCulture_PooledSample1,Set1,P12111,201,2.15e+10,CoCulture_PooledSample,18906047417


I'm getting values on the same order of magnitude, but not exactly the same as those from Proteome Discoverer.

# 2. Detection Filters (Round 1)

Unable to replicate the 1% peptide/ 5% protein FDR, so we'll just retain proteins that were identified by at least 2 peptides.

In [7]:
detection_filter_1_df = normalized_df %>%
    filter(Unique_Peptides >= 2) %>%
    select(-Unique_Peptides) 
head(detection_filter_1_df)

#number of proteins kept
length(unique(detection_filter_1_df$Protein))

ID,Set,Protein,Value,Treatment,Norm_Value
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
CoCulture_PooledSample1,Set1,P02769,4.89e+11,CoCulture_PooledSample,430002659858
CoCulture_PooledSample1,Set1,P02751,2.73e+11,CoCulture_PooledSample,240062834645
CoCulture_PooledSample1,Set1,P12763,2.63e+11,CoCulture_PooledSample,231269324218
CoCulture_PooledSample1,Set1,Q2UVX4,5.65e+10,CoCulture_PooledSample,49683333910
CoCulture_PooledSample1,Set1,P98160,2.27e+10,CoCulture_PooledSample,19961268668
CoCulture_PooledSample1,Set1,P12111,2.15e+10,CoCulture_PooledSample,18906047417


[1] 1814

# 3. Log2 Transformation

Did this step later in the imputation functions. 

# 4. Detection Filters (Round 2)

Filter rows based on valid values: 50 % valid in at least one treatment group (i.e. proteins with values observed in fewer than 50% of replicates (here, 3 reps) in at least 1 condition were eliminated from the data set) 

In [8]:
removed_proteins_df = proteome_df %>%
    #creating 2 count cols: 1st for non-missing data, 2nd for all data
    mutate(data_count = ifelse(is.na(Value), 0, 1), all_count = 1) %>%
    group_by(Set, Treatment, Protein) %>%
    summarize(Protein_Presence = sum(data_count)/ sum(all_count) * 100) %>%
    filter(Protein_Presence < 50) %>%
    arrange(Protein_Presence) %>%
    #just want a column that specifies the treatment and exposure separately to more easily remove proteins later
    separate(Treatment, c("Tx", "Exposure"), sep = "_") %>%
    select(-Exposure)

head(removed_proteins_df)

removed_proteins = unique(removed_proteins_df$Protein)
#number of proteins removed
length(removed_proteins)

`summarise()` has grouped output by 'Set', 'Treatment'. You can override using
the `.groups` argument.


Set,Tx,Protein,Protein_Presence
<chr>,<chr>,<chr>,<dbl>
Set1,CoCulture,O00142,0
Set1,CoCulture,O00571,0
Set1,CoCulture,O00629,0
Set1,CoCulture,O15231,0
Set1,CoCulture,O43405,0
Set1,CoCulture,O43488,0


[1] 453

In [9]:
# removing those 453 proteins from the entire dataset (but within each set)

#modifying cols of detection filter 1 df
detection_filter_1_df = detection_filter_1_df %>%
    mutate(Tx = Treatment) %>%    
    separate(Tx, c("Tx", "Exposure"), sep = "_")

#finding proteins that exist only in the detection_filter_1_df but not the removed proteins_df
detection_filter_2_df = anti_join(detection_filter_1_df, removed_proteins_df)
head(detection_filter_2_df)

#number of proteins kept
length(unique(detection_filter_2_df$Protein))

Joining, by = c("Set", "Protein", "Tx")


ID,Set,Protein,Value,Treatment,Norm_Value,Tx,Exposure
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<chr>,<chr>
CoCulture_PooledSample1,Set1,P02769,4.89e+11,CoCulture_PooledSample,430002659858,CoCulture,PooledSample
CoCulture_PooledSample1,Set1,P02751,2.73e+11,CoCulture_PooledSample,240062834645,CoCulture,PooledSample
CoCulture_PooledSample1,Set1,P12763,2.63e+11,CoCulture_PooledSample,231269324218,CoCulture,PooledSample
CoCulture_PooledSample1,Set1,Q2UVX4,5.65e+10,CoCulture_PooledSample,49683333910,CoCulture,PooledSample
CoCulture_PooledSample1,Set1,P98160,2.27e+10,CoCulture_PooledSample,19961268668,CoCulture,PooledSample
CoCulture_PooledSample1,Set1,P12111,2.15e+10,CoCulture_PooledSample,18906047417,CoCulture,PooledSample


[1] 1726

# 5. QRILC Imputation

QRILC replaces data with left-censored data from a Gaussian distribution. 

In [10]:
# all the missing data was removed for the peptide normalization step, but we need to get that missing data back 
# since that's the data that will be imputed

#first removing proteins that didn't pass the second background filter
missing_data_df = anti_join(proteome_df %>%
    select(-Unique_Peptides), removed_proteins_df) %>%
    #now only keeping missing data 
    #for some reason this protein had 2 entries, so removing the entries with all the missing data
    filter(is.na(Value), Protein != "P08727")

preimputed_df = full_join(missing_data_df %>%
                          mutate(Norm_Value = NA), detection_filter_2_df)

#changing col order to make later steps easier
preimputed_df = preimputed_df[,c(1,2,5,3,4,6)]

head(preimputed_df)

Joining, by = c("Set", "Protein")
Joining, by = c("ID", "Set", "Protein", "Value", "Treatment", "Norm_Value")


,ID,Set,Treatment,Protein,Value,Norm_Value
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,CoCulture_PooledSample1,Set1,CoCulture_PooledSample,H-INV:HIT000292931,NA,NA
2,CoCulture_PooledSample1,Set1,CoCulture_PooledSample,Q6NT21,NA,NA
3,CoCulture_PooledSample1,Set1,CoCulture_PooledSample,P19012,NA,NA
4,CoCulture_PooledSample1,Set1,CoCulture_PooledSample,O43897,NA,NA
5,CoCulture_PooledSample1,Set1,CoCulture_PooledSample,Q15435,NA,NA
6,CoCulture_PooledSample1,Set1,CoCulture_PooledSample,Q07092,NA,NA


In [11]:
#first splitting df to impute within set and treatment (not imputing pooled treatment groups)
preimputed_df_split = preimputed_df %>%
    group_by(Set, Treatment) %>%
    group_split()

coculutre_exp_preimputed_df = preimputed_df_split[[1]]
coculutre_pooled_preimputed_df = preimputed_df_split[[2]]
coculutre_vehicleCT_preimputed_df = preimputed_df_split[[3]] %>%
    #for some reason this protein had another duplicate value, so removing it here
    filter(Protein != "P08727" | Value != 17368304)
triculutre_exp_preimputed_df = preimputed_df_split[[4]]
triculutre_pooled_preimputed_df = preimputed_df_split[[5]]
triculutre_vehicleCT_preimputed_df = preimputed_df_split[[6]]

In [14]:
length(unique(coculutre_exp_preimputed_df$Protein))

[1] 1286

In [32]:
# QRILC
QRILC_imputation = function(dataset){
    wider_dataset = dataset %>%
        select(-c("Set", "Value", "Treatment")) %>%
        pivot_wider(names_from = Protein, values_from = Norm_Value) %>%
        select(-ID)
    
    #normalizing data since that's what the QRILC function wants
    QRILC_prep = wider_dataset %>%
         mutate_all(., function(x) log2(x)) %>%
         as.matrix()

    imputed_QRILC_object = impute.QRILC(QRILC_prep, tune.sigma = 0.1)
    QRILC_log2_df = data.frame(imputed_QRILC_object[1]) 
    
    #converting back the original scale
    QRILC_df = QRILC_log2_df %>%
        mutate_all(., function(x) 2^x - 1)
            
    #adding back in ID, Tx, and Set columns
    final_pivoted_column_number = dim(QRILC_df)[2] + 3
    imputed_dataset = data.frame(cbind(dataset[1:6,1:3], QRILC_df)) %>%
        pivot_longer(cols = 4:all_of(final_pivoted_column_number), names_to = "Protein", values_to = "Value")
    
    return(imputed_dataset)
}

# calling fn
coculutre_exp_imputed_df = QRILC_imputation(coculutre_exp_preimputed_df)
coculutre_vehicleCT_imputed_df = QRILC_imputation(coculutre_vehicleCT_preimputed_df)
triculutre_exp_imputed_df = QRILC_imputation(triculutre_exp_preimputed_df)
triculutre_vehicleCT_imputed_df = QRILC_imputation(triculutre_vehicleCT_preimputed_df)

Warning message in qnorm(seq((pNAs + 0.001), (upper.q + 0.001), (upper.q - pNAs)/(upper.q * :
“NaNs produced”


ERROR: Error in lm.fit(x, y, offset = offset, singular.ok = singular.ok, ...): 0 (non-NA) cases


In [13]:
imputed_df = rbind(coculutre_exp_imputed_df, coculutre_vehicleCT_imputed_df, triculutre_exp_imputed_df,
                  triculutre_vehicleCT_imputed_df)
head(imputed_df)

ID,Set,Treatment,Protein,Value
<chr>,<chr>,<chr>,<chr>,<dbl>
CoCulture_Exposed1,Set1,CoCulture_Exposed,P19012,476617.6
CoCulture_Exposed1,Set1,CoCulture_Exposed,P19013,1077007.5
CoCulture_Exposed1,Set1,CoCulture_Exposed,O43897,2245997.6
CoCulture_Exposed1,Set1,CoCulture_Exposed,P20700,2388716.6
CoCulture_Exposed1,Set1,CoCulture_Exposed,P22102,1790626.6
CoCulture_Exposed1,Set1,CoCulture_Exposed,O75367,874143.2


# 6. PCA

Using PCA to identify and remove outliers.

In [ ]:
pca_df = imputed_df %>%
    pivot_wider(col = )

In [14]:
#exporting results
write.csv(imputed_df, paste0(Output,"/", cur_date, "_Imputed_Proteomics_Data.csv"), row.names = FALSE)